In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta
import time
import base64
from IPython.display import HTML
from tqdm import tqdm


API_KEY = "RpaX4cXYXe5uxWJzOmZyxMwuj9k4CQlY"
TICKER = "NVDA"
ARTICLE_CAP_PER_DAY = 30


DATE_RANGES = [
    ("2022-09-01", "2023-09-01"),
    ("2023-09-02", "2024-09-01"),
    ("2024-09-02", "2025-09-01")
]

print(f" Starting 'Safe Mode' Fetch for {TICKER} (3-Year History)...")
print("   NOTE: We will sleep for 20 seconds between years to respect API limits.")

all_articles = []


for start_date, end_date in DATE_RANGES:
    print(f"\nFetching Batch: {start_date} to {end_date}...")

    base_url = "https://api.polygon.io/v2/reference/news"
    params = {
        "ticker": TICKER,
        "published_utc.gte": start_date,
        "published_utc.lte": end_date,
        "limit": 1000,
        "sort": "published_utc",
        "order": "desc",
        "apiKey": API_KEY
    }

    current_url = base_url
    batch_articles = []

    while True:
        if current_url == base_url:
            response = requests.get(current_url, params=params)
        else:
            response = requests.get(f"{current_url}&apiKey={API_KEY}")

        if response.status_code == 429:
            print("  Rate Limit Hit! Sleeping 60s...")
            time.sleep(60)
            continue

        data = response.json()

        if response.status_code == 200:
            results = data.get('results', [])
            batch_articles.extend(results)
            print(f"   -> Got {len(results)} articles (Batch Total: {len(batch_articles)})")

            if 'next_url' in data:
                current_url = data['next_url']
                time.sleep(0.5)
            else:
                break
        else:
            print(f" Error: {response.status_code}")
            break

    all_articles.extend(batch_articles)
    print(f" Finished Year. Total Articles so far: {len(all_articles)}")
    print(" Resting for 20 seconds to reset API quota...")
    time.sleep(20)


if len(all_articles) > 0:
    df = pd.DataFrame(all_articles)
    df['Date'] = pd.to_datetime(df['published_utc']).dt.date


    df_capped = df.groupby('Date').head(ARTICLE_CAP_PER_DAY).copy()


    df_capped['Source'] = df_capped['publisher'].apply(lambda x: x.get('name') if isinstance(x, dict) else 'Unknown')
    df_clean = df_capped[['Date', 'title', 'article_url', 'Source', 'description']].rename(columns={
        'title': 'Headline', 'article_url': 'Link', 'description': 'Summary'
    })


    full_date_range = pd.date_range(start="2022-09-01", end="2025-09-01").date
    df_dates = pd.DataFrame(full_date_range, columns=['Date'])

    df_final = pd.merge(df_dates, df_clean, on='Date', how='left')
    df_final['Headline'] = df_final['Headline'].fillna("No News")
    df_final['Summary'] = df_final['Summary'].fillna("Market Closed or Quiet Day")
    df_final['Source'] = df_final['Source'].fillna("None")


    filename = f"{TICKER}_Sept_2022_to_Sept_2025_News_COMPLETE.csv"
    df_final.to_csv(filename, index=False)

    print("\n" + "="*50)
    print(f"🎉 SUCCESS! Downloaded {len(df_final)} rows.")
    print("="*50)


    with open(filename, 'rb') as f:
        b64 = base64.b64encode(f.read()).decode()
    payload = f'data:text/csv;base64,{b64}'
    html = f'<a download="{filename}" href="{payload}" target="_blank"><button style="background-color:#4CAF50;color:white;padding:10px;">DOWNLOAD COMPLETE 3-YEAR CSV</button></a>'
    display(HTML(html))
else:
    print(" Failed to fetch any articles.")

In [ ]:

# STAGE 2: GPU-OPTIMIZED
!pip install transformers torch pandas tqdm datasets

import pandas as pd
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
from tqdm.auto import tqdm
import os

INPUT_FILE = "NVDA_Sept_2022_to_Sept_2025_News_COMPLETE.csv"
OUTPUT_FILE = "NVDA_3Year_Processed_Data.csv"
BATCH_SIZE = 64

device = 0 if torch.cuda.is_available() else -1
print(f" Hardware Check: {'GPU ONLINE' if device==0 else ' CPU ONLY'}")

if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE)
    df = df.dropna(subset=['Headline', 'Summary'])

    df['Headline'] = df['Headline'].astype(str)
    df['Summary'] = df['Summary'].astype(str)

    print(f" Loaded {len(df)} articles.")
else:
    print(f" Error: '{INPUT_FILE}' not found.")
    df = pd.DataFrame()

if not df.empty:
    print("\n Loading AI Models...")

    df['AI_Text'] = df['Headline'] + ". " + df['Summary']

    hf_dataset = Dataset.from_pandas(df[['AI_Text']])

    sentiment_pipe = pipeline(
        "text-classification",
        model="ProsusAI/finbert",
        device=device,
        batch_size=BATCH_SIZE,
        truncation=True,
        max_length=512
    )

    relation_pipe = pipeline(
        "zero-shot-classification",
        model="valhalla/distilbart-mnli-12-3",
        device=device,
        batch_size=BATCH_SIZE
    )
    RELATION_LABELS = ["Competitor", "Supplier", "Customer", "Partner", "Regulatory", "Neutral"]

    print(f"\n⚡ Streaming {len(df)} articles to GPU...")

    print("   ... Calculating Sentiment")
    sent_scores = []

    for out in tqdm(sentiment_pipe(KeyDataset(hf_dataset, "AI_Text")), total=len(df)):
        score = out['score'] if out['label'] == 'positive' else -out['score'] if out['label'] == 'negative' else 0.0
        sent_scores.append(score)

    print("   ... Calculating Relationships")
    rel_types = []

    for out in tqdm(relation_pipe(KeyDataset(hf_dataset, "AI_Text"), candidate_labels=RELATION_LABELS), total=len(df)):
        rel_types.append(out['labels'][0])

    df['Sentiment_Score'] = sent_scores
    df['Relation_Type'] = rel_types

    df_final = df[['Date', 'Headline', 'Sentiment_Score', 'Relation_Type']]
    df_final.to_csv(OUTPUT_FILE, index=False)

    print(f"\n SUCCESS! Processed 3 Years of Data.")
    print(f"   Saved to: {OUTPUT_FILE}")

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import pickle

input_file = 'NVDA_3Year_Processed_Data.csv'
output_file = 'NVDA_Dynamic_Graph.pkl'

relation_types = ['Ego', 'Competitor', 'Regulatory', 'Partner', 'Supplier', 'Customer', 'Neutral']
rel_to_idx = {r: i for i, r in enumerate(relation_types)}
num_relations = len(relation_types)

def create_knowledge_graph(csv_path, pkl_path):
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)

    grouped = df.groupby('Date')

    dataset = []

    print("Processing graphs...")
    for date, group in grouped:

        num_news = len(group)
        num_nodes = 1 + num_news

        x = torch.zeros((num_nodes, num_relations + 1), dtype=torch.float)

        x[0, rel_to_idx['Ego']] = 1.0
        x[0, -1] = 0.0

        for i, (_, row) in enumerate(group.iterrows()):
            node_idx = i + 1
            rel_type = row['Relation_Type']
            sentiment = row['Sentiment_Score']

            if rel_type in rel_to_idx:
                x[node_idx, rel_to_idx[rel_type]] = 1.0

            x[node_idx, -1] = sentiment

        sources = torch.arange(1, num_nodes, dtype=torch.long)
        targets = torch.zeros(num_news, dtype=torch.long)

        edge_index = torch.stack([sources, targets], dim=0)

        data = Data(x=x, edge_index=edge_index)

        data.date = date
        data.num_nodes = num_nodes

        dataset.append(data)

    print(f"Saving {len(dataset)} graphs to {pkl_path}...")
    with open(pkl_path, 'wb') as f:
        pickle.dump(dataset, f)

    print("Done!")

if __name__ == "__main__":
    create_knowledge_graph(input_file, output_file)

In [ ]:
import pickle
import torch
from torch_geometric.data import Data

with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
    graph_dicts = pickle.load(f)

dataset = []
for g in graph_dicts:
    x = torch.tensor(g['x'], dtype=torch.float)
    edge_index = torch.tensor(g['edge_index'], dtype=torch.long)

    data = Data(x=x, edge_index=edge_index)

    data.date = g['date']

    dataset.append(data)

print(f"Successfully loaded {len(dataset)} daily graph snapshots.")
print(dataset[0])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import pickle


# Define the GCN Model

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(num_node_features, hidden_channels)

        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):

        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

print("Loading graph data...")
with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
    dataset = pickle.load(f)

sample_graph = dataset[0]
num_node_features = sample_graph.x.shape[1]

print(f"Loaded {len(dataset)} graphs.")
print(f"Feature size per node: {num_node_features}")

HIDDEN_CHANNELS = 16
NUM_CLASSES = 2

model = GCN(num_node_features, HIDDEN_CHANNELS, NUM_CLASSES)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
sample_graph = sample_graph.to(device)

print("\nModel Architecture:")
print(model)

model.eval()
with torch.no_grad():
    out = model(sample_graph)

print("\nOutput shape from first graph:", out.shape)
print("First 5 node predictions (Log Softmax):\n", out[:5])

In [ ]:
# WALK-FORWARD VALIDATION

!pip install torch_geometric > /dev/null
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import pickle
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from dateutil.relativedelta import relativedelta

TICKER = "NVDA"
START_DATE = "2022-09-01"
END_DATE = "2025-09-02"

WALK_FORWARD_START = "2024-01-01"

SEQ_LEN = 5
HIDDEN_DIM = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Processing on: {DEVICE}")

print("\nLoading Knowledge Graph...")
try:
    with open('NVDA_Dynamic_Graph.pkl', 'rb') as f:
        raw_graph_data = pickle.load(f)
except FileNotFoundError:
    raise FileNotFoundError("Error: 'NVDA_Dynamic_Graph.pkl' not found.")

graph_map = {}
for item in raw_graph_data:
    x = item['x'].clone().detach().float()
    edge_index = item['edge_index'].clone().detach().long()
    data = Data(x=x, edge_index=edge_index)
    d_str = pd.to_datetime(item['date']).strftime('%Y-%m-%d')
    graph_map[d_str] = data

# 2. FINANCIAL DATA
print(f"\n Fetching Stock Data...")
df = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)
if isinstance(df.columns, pd.MultiIndex): df.columns = [c[0] for c in df.columns]
df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

df['Close'] = df['Close'].replace(0, np.nan).ffill()
df['Log_Ret'] = np.log(df['Close'] / df['Close'].shift(1))
df['Volatility'] = df['Log_Ret'].rolling(window=20).std()
df['Target_Return'] = df['Log_Ret'].shift(-1)
df.dropna(inplace=True)

KPI_COLS = ['Log_Ret', 'Volatility']

scaler = StandardScaler()
df[KPI_COLS] = scaler.fit_transform(df[KPI_COLS])

date_to_feats  = {row['Date']: row[KPI_COLS].values.astype(float) for _, row in df.iterrows()}
date_to_target = {row['Date']: row['Target_Return'] for _, row in df.iterrows()}
valid_dates = sorted(list(set(date_to_feats.keys()) & set(graph_map.keys())))

class InstitutionalTrader(nn.Module):
    def __init__(self, node_dim, kpi_dim, hidden_dim=64):
        super().__init__()
        self.gnn = GCNConv(node_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim + kpi_dim, hidden_dim, batch_first=True)
        self.head = nn.Linear(hidden_dim, 1)

    def forward(self, graph_list, kpi_tensor):
        graph_embeds = []
        for g in graph_list:
            x, edge_index = g.x.to(DEVICE), g.edge_index.to(DEVICE)
            x = torch.tanh(self.gnn(x, edge_index))
            graph_embeds.append(global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long).to(DEVICE)))

        fusion = torch.cat((torch.stack(graph_embeds, dim=1), kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

def create_sequences(dates_list):
    seqs, targets = [], []
    for i in range(len(dates_list) - SEQ_LEN):
        window_dates = dates_list[i : i+SEQ_LEN]
        target_date  = dates_list[i + SEQ_LEN]
        if target_date not in date_to_target: continue
        seqs.append(([graph_map[d] for d in window_dates], [date_to_feats[d] for d in window_dates]))
        targets.append(date_to_target[target_date])
    return seqs, targets

print(f"\nSTARTING WALK-FORWARD VALIDATION (From {WALK_FORWARD_START})...")

current_date = pd.to_datetime(WALK_FORWARD_START)
end_date_dt = pd.to_datetime(df['Date'].max())

all_predictions = []
all_targets = []

while current_date < end_date_dt:
    next_month = current_date + relativedelta(months=1)

    cutoff_str = current_date.strftime('%Y-%m-%d')
    next_cutoff_str = next_month.strftime('%Y-%m-%d')

    print(f"Training up to {cutoff_str} | Testing {cutoff_str} -> {next_cutoff_str}")

    train_subset = [d for d in valid_dates if d <= cutoff_str]
    test_subset  = [d for d in valid_dates if d > cutoff_str and d <= next_cutoff_str]

    if len(test_subset) == 0:
        break

    train_seqs, train_y = create_sequences(train_subset)
    test_seqs, test_y   = create_sequences(test_subset)
    sample_g = train_seqs[0][0][0]
    NODE_DIM = sample_g.x.shape[1]
    KPI_DIM  = len(KPI_COLS)

    model = InstitutionalTrader(NODE_DIM, KPI_DIM, HIDDEN_DIM).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    model.train()
    for _ in range(10):
        for i in range(len(train_seqs)):
            gs, kpis = train_seqs[i]
            tgt = torch.tensor([[train_y[i]]]).float().to(DEVICE)
            kpi_in = torch.tensor(np.array(kpis)).float().unsqueeze(0).to(DEVICE)
            optimizer.zero_grad()
            loss_fn(model(gs, kpi_in), tgt).backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        for i in range(len(test_seqs)):
            gs, kpis = test_seqs[i]
            kpi_in = torch.tensor(np.array(kpis)).float().unsqueeze(0).to(DEVICE)
            pred = model(gs, kpi_in).item()

            all_predictions.append(pred)
            all_targets.append(test_y[i])

    current_date = next_month

print("\n" + "="*40)
print(f"WALK-FORWARD RESULTS")
print("="*40)

preds_arr = np.array(all_predictions)
targs_arr = np.array(all_targets)

strat_returns = np.where(preds_arr > 0, targs_arr, 0.0)

if np.std(strat_returns) == 0: sharpe = 0
else: sharpe = (np.mean(strat_returns) / np.std(strat_returns)) * np.sqrt(252)

cum_ret = np.cumsum(strat_returns)
peak = np.maximum.accumulate(cum_ret)
drawdown = np.min(cum_ret - peak) if len(cum_ret) > 0 else 0
total_return = np.sum(strat_returns) * 100

print(f"Total Test Days: {len(strat_returns)}")
print(f"Sharpe Ratio:    {sharpe:.3f}")
print(f"Max Drawdown:    {drawdown:.3f}")
print(f"Total Return:    {total_return:.2f}% (Log Scale approx)")

if sharpe > 1.0:
    print("VERDICT: Robust Institutional Strategy")
else:
    print("VERDICT: Strategy failed Walk-Forward test")

In [ ]:
# VISUALIZATION

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

preds_arr = np.array(all_predictions)
targs_arr = np.array(all_targets)

strat_log_ret = np.where(preds_arr > 0, targs_arr, 0.0)
hold_log_ret  = targs_arr

initial_capital = 10000
equity_strat = initial_capital * np.exp(np.cumsum(strat_log_ret))
equity_hold  = initial_capital * np.exp(np.cumsum(hold_log_ret))

running_max_strat = np.maximum.accumulate(equity_strat)
drawdown_strat = (equity_strat - running_max_strat) / running_max_strat

test_dates = pd.to_datetime(df['Date'].values[-len(preds_arr):])

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)

plt.plot(test_dates, equity_hold, label='Buy & Hold (NVDA)', color='gray', alpha=0.5, linestyle='--')
plt.plot(test_dates, equity_strat, label='Model Strategy (Walk-Forward)', color='#00d2be', linewidth=2)

plt.title(f"Performance Comparison: Model vs. Buy & Hold ({len(test_dates)} Days)", fontsize=14)
plt.ylabel("Portfolio Value ($)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.subplot(2, 1, 2)
plt.fill_between(test_dates, drawdown_strat * 100, 0, color='red', alpha=0.3, label='AI Drawdown')
plt.plot(test_dates, drawdown_strat * 100, color='red', linewidth=1)

plt.title("Risk Profile: Underwater Plot (Drawdown)", fontsize=12)
plt.ylabel("Drawdown (%)")
plt.xlabel("Date")
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.tight_layout()
plt.show()

In [ ]:
MODEL_FILE = "NVDA_Institutional_Model.pth"
torch.save(model.state_dict(), MODEL_FILE)
print(f"Brain Saved: {MODEL_FILE}")
print("   (The Agent will load this file to make predictions)")

In [ ]:
print("Installing Streamlit, Ngrok, and Graph Neural Networks...")
!pip install streamlit pyngrok plotly yfinance torch_geometric > /dev/null

print("Installation Complete. You can now proceed to run the App.")

In [ ]:
#THE INSTITUTIONAL AGENT (app.py)

%%writefile app.py
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import plotly.graph_objects as go
import os

TICKER = "NVDA"
MODEL_FILE = "NVDA_Institutional_Model.pth"
SEQ_LEN = 5
DEVICE = torch.device('cpu')

class InstitutionalTrader(nn.Module):
    def __init__(self, node_dim, kpi_dim, hidden_dim=64):
        super().__init__()
        self.gnn = GCNConv(node_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim + kpi_dim, hidden_dim, batch_first=True)
        self.head = nn.Linear(hidden_dim, 1)

    def forward(self, graph_list, kpi_tensor):
        graph_embeds = []
        for g in graph_list:
            x, edge_index = g.x.to(DEVICE), g.edge_index.to(DEVICE)
            x = torch.tanh(self.gnn(x, edge_index))
            batch_vec = torch.zeros(x.size(0), dtype=torch.long).to(DEVICE)
            graph_embed = global_mean_pool(x, batch_vec)
            graph_embeds.append(graph_embed)

        fusion = torch.cat((torch.stack(graph_embeds, dim=1), kpi_tensor), dim=2)
        lstm_out, _ = self.lstm(fusion)
        return self.head(lstm_out[:, -1, :])

def get_historical_stock_data(end_date_str):
    """Fetches data and calculates INSTITUTIONAL features (Log Ret, Vol)."""
    end_dt_obj = datetime.strptime(end_date_str, '%Y-%m-%d') + timedelta(days=1)
    query_end = end_dt_obj.strftime('%Y-%m-%d')
    start_dt_obj = end_dt_obj - timedelta(days=120)
    query_start = start_dt_obj.strftime('%Y-%m-%d')

    df = yf.download(TICKER, start=query_start, end=query_end, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df.columns = [c[0] for c in df.columns]
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    df['Close'] = df['Close'].replace(0, np.nan).ffill()
    df['Log_Ret'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Volatility'] = df['Log_Ret'].rolling(window=20).std()

    df.dropna(inplace=True)
    return df[df['Date'] <= end_date_str].copy()

def get_trading_day_before(target_date_obj):
    prev_dt = target_date_obj - timedelta(days=1)
    while prev_dt.weekday() > 4:
        prev_dt -= timedelta(days=1)
    return prev_dt.strftime('%Y-%m-%d')

st.set_page_config(page_title="NVIDIA Agent", layout="wide")
st.title("NVIDIA Alpha Agent")
st.markdown("""
**Strategy:** GCN-LSTM trained on **Log Returns** & **Volatility. Detects regime changes using Walk-Forward Validation logic.**
""")

st.sidebar.header("Simulation Settings")
target_date_input = st.sidebar.date_input("Target Date", datetime.today() + timedelta(days=1))
target_date_str = target_date_input.strftime('%Y-%m-%d')

if st.sidebar.button("Run Strategy"):
    with st.spinner(f"Analyzing Market Microstructure up to {target_date_str}..."):

        decision_date_str = get_trading_day_before(target_date_input)
        df_seq = get_historical_stock_data(decision_date_str)

        if len(df_seq) < SEQ_LEN:
            st.error("Not enough data. Market too volatile or date too old.")
        else:

            KPI_COLS = ['Log_Ret', 'Volatility']
            scaler = StandardScaler()

            df_seq[KPI_COLS] = scaler.fit_transform(df_seq[KPI_COLS])

            seq_kpis = df_seq[KPI_COLS].tail(SEQ_LEN).values
            kpi_tensor = torch.tensor(seq_kpis, dtype=torch.float).unsqueeze(0).to(DEVICE)

            DETECTED_NODES = 50
            x = torch.eye(DETECTED_NODES)
            edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
            graph_snapshot = Data(x=x, edge_index=edge_index)
            graph_seq = [graph_snapshot] * SEQ_LEN

            model = InstitutionalTrader(node_dim=DETECTED_NODES, kpi_dim=len(KPI_COLS)).to(DEVICE)

            if os.path.exists(MODEL_FILE):
                try:
                    state_dict = torch.load(MODEL_FILE, map_location=DEVICE)
                    model_dict = model.state_dict()

                    pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
                    model_dict.update(pretrained_dict)
                    model.load_state_dict(model_dict)
                    model.eval()

                    with torch.no_grad():
                        pred_log_ret = model(graph_seq, kpi_tensor).item()

                except Exception as e:
                    st.warning(f"Model Load Warning: {e}. Using uncalibrated weights.")
                    pred_log_ret = 0.005
            else:
                st.error("Model file not found. Run Block 8.5 first!")
                pred_log_ret = 0.0

            last_close = df_seq['Close'].iloc[-1]
            predicted_price = last_close * np.exp(pred_log_ret)
            pct_change = (np.exp(pred_log_ret) - 1) * 100

            col1, col2, col3 = st.columns(3)
            col1.metric("Last Close", f"${last_close:.2f}", decision_date_str)

            color = "green" if pred_log_ret > 0 else "red"
            col2.metric("AI Forecast", f"${predicted_price:.2f}", f"{pct_change:.2f}%")

            signal = "BUY / LONG" if pred_log_ret > 0 else "CASH / NEUTRAL"
            col3.markdown(f"### Signal: :{color}[{signal}]")

            fig = go.Figure()

            plot_df = df_seq.tail(60)
            fig.add_trace(go.Scatter(x=plot_df['Date'], y=plot_df['Close'], mode='lines', name='Price Action', line=dict(color='#00d2be')))

            fig.add_trace(go.Scatter(x=[target_date_str], y=[predicted_price], mode='markers', name='Forecast', marker=dict(color=color, size=15, symbol='star')))

            fig.add_trace(go.Scatter(x=[plot_df['Date'].iloc[-1], target_date_str], y=[plot_df['Close'].iloc[-1], predicted_price], mode='lines', line=dict(color='gray', dash='dot'), showlegend=False))

            fig.update_layout(template="plotly_dark", height=400, title="Institutional Price Projection")
            st.plotly_chart(fig, use_container_width=True)

            st.success(f"**Logic:** Model predicts a **{pct_change:.2f}%** return based on current Volatility regime and Knowledge Graph sentiment.")

else:
    st.info("Select a date to run the Walk-Forward Inference Engine.")

In [ ]:
import os
import time
import sys

print(" Verifying Streamlit installation...")
!pip install streamlit pyngrok > /dev/null

from pyngrok import ngrok

ngrok.kill()

NGROK_TOKEN = "37AWuu4MjUuB90VQK7gLkm3ptlK_2NCrw2uygxsntWCaZXs3e"
ngrok.set_auth_token(NGROK_TOKEN)

print("Starting Streamlit in Background...")
get_ipython().system_raw(f'{sys.executable} -m streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &')

time.sleep(5)

try:
    public_url = ngrok.connect(8501).public_url
    print(f"AGENT LIVE: {public_url}")
    print("   (If the link errors, wait 10 seconds and reload)")
except Exception as e:
    print(f"Ngrok Error: {e}")


print("\n Checking Application Status...")
if os.path.exists("streamlit.log"):
    with open("streamlit.log", "r") as f:
        logs = f.read()
        if "Error" in logs or "Traceback" in logs:
            print(" CRITICAL APP ERROR FOUND IN LOGS:")
            print(logs[-500:])
        else:
            print("   App seems healthy. Log file created successfully.")